In [7]:

!pip install pymysql
!pip install pandas




[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pymysql
import pandas as pd


In [9]:
# Database connection details
db_config = {
    "host": "localhost",  # Change to your database host
    "user": "root",       # Change to your database username
    "password": "root",  # Change to your database password
    "database": "sales"  # Change to your database name
}

In [12]:
queries = {

    # Question 1: Top 5 Products by Sales in 2022 (Mobiles & Tablets)
    "question_1_top_5_products": """
    SELECT
        skudet.sku_name,
        skudet.category,
        SUM(ordet.qty_ordered) AS total_quantity
    FROM order_detail AS ordet
    JOIN sku_detail AS skudet
      ON ordet.sku_id = skudet.id
    WHERE skudet.category = 'Mobiles & Tablets'
      AND ordet.is_valid = 1
      AND YEAR(ordet.order_date) = 2022
    GROUP BY skudet.sku_name, skudet.category
    ORDER BY total_quantity DESC
    LIMIT 5;
    """,

    # Question 2: Sales Decrease in "Others" Category (2021 vs 2022)
    "question_2_sales_decrease_others": """
    WITH sales_2021 AS (
        SELECT
            skudet.sku_name,
            SUM(ordet.qty_ordered) AS qty_2021
        FROM order_detail ordet
        JOIN sku_detail skudet
          ON ordet.sku_id = skudet.id
        WHERE skudet.category = 'Others'
          AND ordet.is_valid = 1
          AND YEAR(ordet.order_date) = 2021
        GROUP BY skudet.sku_name
    ),
    sales_2022 AS (
        SELECT
            skudet.sku_name,
            SUM(ordet.qty_ordered) AS qty_2022
        FROM order_detail ordet
        JOIN sku_detail skudet
          ON ordet.sku_id = skudet.id
        WHERE skudet.category = 'Others'
          AND ordet.is_valid = 1
          AND YEAR(ordet.order_date) = 2022
        GROUP BY skudet.sku_name
    )
    SELECT
        s21.sku_name,
        s21.qty_2021,
        s22.qty_2022,
        (s22.qty_2022 - s21.qty_2021) AS qty_difference,
        ROUND(((s22.qty_2022 - s21.qty_2021) / s21.qty_2021) * 100, 2) AS percentage_change,
        CASE
            WHEN s22.qty_2022 < s21.qty_2021 THEN 'DOWN'
            WHEN s22.qty_2022 > s21.qty_2021 THEN 'UP'
            ELSE 'FAIR'
        END AS status
    FROM sales_2021 s21
    JOIN sales_2022 s22
      ON s21.sku_name = s22.sku_name
    ORDER BY qty_difference ASC
    LIMIT 20;
    """,

    # Question 3: Checkout Completed but Not Paid (2022)
    "question_3_checkout_not_paid": """
    SELECT DISTINCT
        customer_id
    FROM order_detail
    WHERE is_gross = 1
      AND is_valid = 0
      AND is_net = 0
      AND YEAR(order_date) = 2022;
    """,

    # Question 4: Weekend vs Weekday Sales (Q4 2022)
    "question_4_weekend_vs_weekday": """
    SELECT
        MONTH(order_date) AS month_id,
        MONTHNAME(order_date) AS month_name,
        CASE
            WHEN DAYNAME(order_date) IN ('Saturday','Sunday') THEN 'Weekend'
            ELSE 'Weekday'
        END AS day_type,
        AVG(before_discount) AS avg_daily_sales
    FROM order_detail
    WHERE order_date BETWEEN '2022-10-01' AND '2022-12-31'
    GROUP BY month_id, month_name, day_type;
    """,

    # Question 5: Products with Largest Sales Decrease (2021 vs 2022)
    "question_5_largest_decrease_products": """
    WITH sales_2021 AS (
        SELECT
            skudet.sku_name,
            SUM(ordet.qty_ordered) AS qty_2021
        FROM order_detail ordet
        JOIN sku_detail skudet
          ON ordet.sku_id = skudet.id
        WHERE YEAR(ordet.order_date) = 2021
        GROUP BY skudet.sku_name
    ),
    sales_2022 AS (
        SELECT
            skudet.sku_name,
            SUM(ordet.qty_ordered) AS qty_2022
        FROM order_detail ordet
        JOIN sku_detail skudet
          ON ordet.sku_id = skudet.id
        WHERE YEAR(ordet.order_date) = 2022
        GROUP BY skudet.sku_name
    )
    SELECT
        s21.sku_name,
        s21.qty_2021,
        s22.qty_2022,
        (s22.qty_2022 - s21.qty_2021) AS qty_difference
    FROM sales_2021 s21
    JOIN sales_2022 s22
      ON s21.sku_name = s22.sku_name
    ORDER BY qty_difference ASC
    LIMIT 10;
    """,

    # Question 6: Category Sales Trends (2022)
    "question_6_category_trends": """
    SELECT
        skudet.category,
        MONTH(ordet.order_date) AS month_id,
        MONTHNAME(ordet.order_date) AS month_name,
        SUM(ordet.qty_ordered) AS total_qty
    FROM order_detail ordet
    JOIN sku_detail skudet
      ON ordet.sku_id = skudet.id
    WHERE YEAR(ordet.order_date) = 2022
    GROUP BY skudet.category, month_id, month_name
    ORDER BY month_id;
    """
}


In [13]:
from IPython.display import display

In [14]:
# Connect to the database
try:
    connection = pymysql.connect(**db_config)
    print("Database connection successful!")
    
    # Execute each query and display results
    for question, query in queries.items():
        print(f"\n--- {question} ---")
        df = pd.read_sql(query, connection)
        display(df)  # Display the DataFrame in the notebook
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if connection:
        connection.close()
        print("Database connection closed.")

Database connection successful!

--- question_1_top_5_products ---


C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1002502647.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,sku_name,category,total_quantity
0,IDROID_BALRX7-Gold,Mobiles & Tablets,3000.0
1,IDROID_BALRX7-Jet black,Mobiles & Tablets,93.0
2,Infinix Hot 4-Gold,Mobiles & Tablets,45.0
3,samsung_Grand Prime Plus-Black,Mobiles & Tablets,33.0
4,samsung_Grand Prime Plus-Silver,Mobiles & Tablets,30.0



--- question_2_sales_decrease_others ---


C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1002502647.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


...

,sku_name,qty_2021,qty_2022,qty_difference
0,RB_Dettol Germ Busting Kit-bf,600.0,135.0,-465.0
1,RS_Baklawa 500gm,321.0,54.0,-267.0
2,jn_900g,171.0,30.0,-141.0
3,RS_Soan Papri-250gm,153.0,21.0,-132.0
4,Topline_Waheed Murad His Life and Our Times,138.0,30.0,-108.0
5,vitamin_Ultra Creatine,78.0,6.0,-72.0
6,Telemall_MM-DR-HB-L,69.0,6.0,-63.0
7,RS_Kaju Barfi,87.0,30.0,-57.0
8,MDF 018_Ajwa Dates 250 gm,57.0,3.0,-54.0
9,cr_CASHEW NUT PLAIN-500 GM,66.0,15.0,-51.0



--- question_6_category_trends ---


C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1002502647.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,category,month_id,month_name,total_qty
0,Kids & Baby,1,January,111.0
1,Computing,1,January,81.0
2,Mobiles & Tablets,1,January,174.0
3,Entertainment,1,January,81.0
4,Appliances,1,January,90.0
...,...,...,...,...
175,School & Education,12,December,21.0
176,Kids & Baby,12,December,30.0
177,Soghaat,12,December,60.0
178,Superstore,12,December,90.0


Database connection closed.


In [15]:
import os

In [16]:
# Database connection details
db_config = {
    "host": "localhost",  # Change to your database host
    "user": "root",       # Change to your database username
    "password": "root",  # Change to your database password
    "database": "sales"  # Change to your database name
}

In [23]:
# Folder path to save CSV files
output_folder = r"C:\Users\sakth\Downloads\Google Looker project\dataset"  # Replace with your folder path
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

In [24]:
# List of tables to export
tables = ["order_detail", "sku_detail", "payment_detail","customer_detail"]  # Add your table names here

In [25]:
try:
    # Connect to the database
    connection = pymysql.connect(**db_config)
    print("Database connection successful!")

    for table in tables:
        print(f"Exporting table: {table}")
        
        # SQL query to fetch all data from the table
        query = f"SELECT * FROM {table};"
        
        # Read table data into a DataFrame
        df = pd.read_sql(query, connection)
        
        # Save the DataFrame to a CSV file
        output_file = os.path.join(output_folder, f"{table}.csv")
        df.to_csv(output_file, index=False)
        
        print(f"Table {table} exported successfully to {output_file}.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if connection:
        connection.close()
        print("Database connection closed.")

Database connection successful!
Exporting table: order_detail
Table order_detail exported successfully to C:\Users\sakth\Downloads\Google Looker project\dataset2\order_detail.csv.
Exporting table: sku_detail
Table sku_detail exported successfully to C:\Users\sakth\Downloads\Google Looker project\dataset2\sku_detail.csv.
Exporting table: payment_detail
Table payment_detail exported successfully to C:\Users\sakth\Downloads\Google Looker project\dataset2\payment_detail.csv.
Exporting table: customer_detail


C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1590211234.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1590211234.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1590211234.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\sakth\AppData\Local\Temp\ipykernel_1776\1590211234.py:13: UserWarning: pa

Table customer_detail exported successfully to C:\Users\sakth\Downloads\Google Looker project\dataset2\customer_detail.csv.
Database connection closed.


In [31]:
df_od = pd.read_csv("order_detail.csv")
df_sd = pd.read_csv("sku_detail.csv")
df_pd = pd.read_csv("payment_detail.csv")
df_cd = pd.read_csv("customer_detail.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'order_detail.csv'